# Mario Level Generator - Sampler

## Library Import

We begin by importing the requirements for our project. You may add any libraries that you wish, such as tensorflow or PyTorch. The example in this notebook uses keras for the machine learning code, but if you are experienced with something else, you are welcome to use whatever is your preference. To add a library, open a terminal and install whatever you require with pip.

In [ ]:
import os, sys
import numpy as np
from collections import Counter
from typing import List
from machina.challenge import helper, score
from IPython.display import Video

## Loading Training Data

For this approach, we'll start by loading in all of the existing levels from the categories of our choice. To do so, we first write this helper function to process all of the level files of our choice and format them for use in our generator. As part of the processing, we are going to convert the levels from the format that the loading function loads them in (a list of strings representing the rows in the level, from top to bottom) to a list of strings representing the columns in the level, with the first string being the left-most column in the level.

In [ ]:
def prepare_training(level_category: str) -> list:
    """
    This function loads all of the levels for a given category of levels. It also converts each from the 
    list-of-rows format in which they load from file to a list-of-columns format. 
    :param level_category: The string name of one of the existing level categories.
    """
    x_train = []
    for level_path in helper.levels_for_category(level_category, full_paths=True):
        # First we load the level from file.
        level = helper.load_level_file(level_path)
        # Next we are going to transpose the level from list-of-rows to list-of-columns format.
        # We'll do this using the library function score.transpose_level. 
        # Run `? score.transpose_level` for more information about this function.
        transposed_level = score.transpose_level(level)
        # Finally, we add the transposed level to the train
        x_train.append(transposed_level)
    return x_train

To see all of the level categories, you can run `helper.level_categories()`. You can then update the `USED_CATEGORIES` list in the cell below to include only the ones that you want. Some categories of levels may be of higher quality than others, so experimenting with different combinations may lead to better performance in your submission.

In [ ]:
print(f'Level categories:\n\t{", ".join(helper.level_categories())}')
# Update this list with options from the level categories list to alter your list of used level examples.
USED_CATEGORIES = [
    'ge',
    'hopper',
    'notch',
    'notchParam',
    'notchParamRand',
    'ore',
    'patternCount',
    'patternOccur',
    'patternWeightCount'
]
print(f'Used categories:\n\t{", ".join(USED_CATEGORIES)}')

# Collect the example levels we want to use in our generator.
example_levels = []
for level_category in USED_CATEGORIES:
    example_levels += prepare_training(level_category)

num_examples = len(example_levels)
print(f'Number of example levels: {num_examples}')

## Build a sampler

The sampler approach to level generation is fairly straightforward. All wee need to do is take chunks of `SAMPLE_WIDTH` from existing levels and piece them together until we have a level of length `OUTPUT_LENGTH`. So, for example, our new level may have the first 10 columns from one level, the next 10 from another, and so on until we reach our desired length. We will implement this as the `sampler` function below.

In [ ]:
def sampler(examples: List[List[str]], sample_width: int, output_length: int = 200) -> List[str]:
    """
    This function generates a new level by taking chunks from existing levels and 
    piecing them together.
    :param examples: Our example_levels, a list of lists of level column-strings.
    :param sample_width: An integer width of the chunks to take from our example 
        levels to put into our new level.
    :param output_length: The final length of the new level that we are creating. 
        For the competition, we want this to be 200, but you can experiment with 
        others for fun.
    :return: A new level in list of row-strings format.
    """
    # Create an empty list to fill with the column strings for our new level.
    new_level = []
    
    current_index = 0
    # We will build our level from block until there is not room for another full block.
    while current_index + sample_width < output_length:
        # Select a random level froom our list of example levels.
        random_level_index = np.random.choice(num_examples)
        random_level = example_levels[random_level_index]
        # Extract a block from that level at our current index.
        block = random_level[current_index:current_index+sample_width]
        # Add that block to the level that we are building.
        new_level += block
        # Jump our position to where the next block needs to be added.
        current_index += sample_width
    
    
    # If we stopped short of a level of exactly output_length, we need to fill out the rest of the level. 
    if current_index != output_length - 1:
        # Pick another random level.
        random_level_index = np.random.choice(num_examples)
        random_level = example_levels[random_level_index]
        # Grab the end of the level to add to ours.
        block = random_level[current_index:]
        # Add it to ours.
        new_level += block

    # Now we have a new level of length output_length which is in list of column-strings format.
    # Most of the helper functions for the competition, however, operate on list of row-strings,
    # so let's flip the level back to that representation before returning it.
    new_level = score.transpose_level(new_level)
    
    return new_level
        

## Generate and save a level

Now that we have a generator function, we can generate and save a level.

In [ ]:
SAMPLE_WIDTH = 10
OUTPUT_LENGTH = 200

new_level_file = os.path.join(os.getcwd(), 'output/sampler-lvl.txt')
sampled = sampler(example_levels, SAMPLE_WIDTH, OUTPUT_LENGTH)
helper.write_level_to_file(sampled, new_level_file)

## Run The Level

Here we will run the level we generated to see how it looks. This will use the `helper` library function `run_level`. For details on this function's use, you can run the following in a code cell:

`? helper.runLevel`

Configured as we have below, `runLevel` will run an agent through our decoded level and then produce a video of the resulting run at the location `output/sampler-lvl.mp4`, relative to the location of this notebook.

In [ ]:
new_level_video_file = os.path.join(os.getcwd(), 'output', 'sampler-lvl.mp4')
print(helper.runLevel(new_level_file, True, new_level_video_file, False))
print(helper.score_level(new_level_file))

Once the run is complete, we can use the `Video` function as below to watch our agent's run through the level.

In [ ]:
Video(new_level_video_file, embed=True)

## Submitting A Generator

In order to submit your generator to the competition, you must create a function like the one below which can be called with no arguments that produces a new level in list of row-strings format.

In [ ]:
def generate_level():
    return sampler(example_levels, SAMPLE_WIDTH, OUTPUT_LENGTH)

### Evaluating your generator

Before you submit your generator, you can first check to see approximately how well it performs. To do so, you can run the evaluation function that will be run at submission time. You will, however, want to run the function multiple times, as evaluation involves scoring several levels generated using your generation function at random. As a result, the evaluation function will not return the same score every time, thus you will want to see how well your generator performs on average.

This process can take a bit of time, as evaluating a generator requires running an agent through each generated level, possibly several times.

In [ ]:
NUM_EVALS = 3

eval_scores = []
for i in range(1, NUM_EVALS+1):
    run_score = helper.evaluate_generator(generate_level)
    eval_scores.append(run_score)
    print(f'Eval {i} score: {run_score}')
    
print(f'Average Score: {round(sum(eval_scores)/NUM_EVALS, 2)}')

### Submitting

When you're happy with your generator, the last step is to submit it to the competition. To do so, you will use the `helper.submit` function. This function takes two arguments:
1. An absolute file path location of this notebook.
2. Your generator function created above. As noted, the generator function must be a function which may be called with no parameters passed to it and returns a level in list of strings format.

Your generator will then be used to generate some new levels and scored based on those levels. It is worth noting that you are free to submit as many times as you like, but each new submission will overwrite all previous ones.

In [ ]:
# Get the path to this notebook.
notebook_path = os.path.join(os.getcwd(), 'Sampler.ipynb')
# Submit to the competition.
helper.submit(notebook_path, generate_level)